# Importings

In [ ]:
import pandas as pd
import json
import folium
import geopandas as gpd
import branca.colormap as cm
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_predict, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_curve,
    roc_auc_score,
    auc,
    precision_recall_fscore_support,
)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
import shap
from sklearn.preprocessing import StandardScaler

# pip install lime
# import lime
# from lime import lime_tabular
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant

import statsmodels.api as sm
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#  pip install xgboost
import xgboost
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Loading Dataset 

In [ ]:
dfdom = pd.read_csv("NDOM_FINAL.csv")

In [ ]:
dfdom.groupby("NUM_CIR")[["IMPORTO_CONTRATTO", "IMPORTO_PAREGGIO"]].agg(
    ["count", "sum", "median", "mean", "min", "max"]
)

### Build a new feature to transform the problem into a classification task

In [ ]:
dfdom["par/em"] = dfdom["IMPORTO_PAREGGIO"] / dfdom["IMPORTO_CONTRATTO"]

### Drop unrelevant features

In [ ]:
dfdom.columns

In [ ]:
dfdom = dfdom.drop(
    columns=[
        "Unnamed: 0",
        "CNT_COD",
        "CONTO",
        "NUM_FATT",
        "UTZ_CONTRATTO",
        "DATA_EMISSIONE",
        "IMPORTO_CONTRATTO",
        "MOTIVO_PAREGGIO",
        "DATA_PAREGGIO",
        "IMPORTO_PAREGGIO",
        "TIPO_DOC_PAREGGIO",
        "CNT_COGNOME",
        "CNT_NOME",
        "CNT_RAG_SOC",
        "CNT_PAR_IVA",
        "CNT_COD_FSC",
        "UTZ_TARIFTYP",
        "UTZ_VIA_DES",
        "UTZ_NUM_CIV",
        "CAP",
        "COD_ZON_URB",
        "FLAG_DECEDUTO",
        "FLAG_CESSATA",
        "RI_CLEAN_STA_PAR_IVA",
        "FLAG_PEC_SAP",
        "SEMESTRE",
        "FLAG_PEC_INV",
        "FLAG_NOPEC",
        "FLAG_PEC",
        "FLAG_ESENTE_SAP",
        "FLAG_ESENTE_RIC",
        "FLAG_ESENTE",
        "FLAG_RATE_SAP",
        "NUM_RATE",
        "IMPORTO_RATEIZZATO",
        "FLAG_RATE_RIC",
        "FLAG_RATE",
        "FLAG_BOLLETTINO",
        "FLAG_DATI_ERRATI",
        "IMPORTO_CONTRATTO_NO_TEFA",
        "IMPORTO_PAREGGIO_NO_TEFA",
        "FLAG_NON_RECAPITATO",
        "PERCENTUALE_NON_PAGATO",
        "FLAG_PAGATO",
        "FLAG_COPIA_CONFORME",
    ]
)

### we have 260455:0  941030:1 8:nan, 529 values that have partially paid amounts.

### We consider not a numbers as paid, because are divisions by zero.

# Construct the class label

In [ ]:
target = dfdom["par/em"]  # class label

In [ ]:
np.sum(target)  # the ones are the paid

In [ ]:
target = target.replace(np.nan, 1.0)

In [ ]:
for i in range(len(target)):
    if (target[i] != 1.0) and (target[i] != 0.0):
        target[i] = 0.0

In [ ]:
np.sum(target)

In [ ]:
dfdom["target"] = target

In [ ]:
dfdom = dfdom.dropna().reset_index(drop=True)

In [ ]:
dfdom = dfdom.drop(columns="par/em")

In [ ]:
len(np.unique(dfdom["SUPERFICIE"], return_counts=True)[0])  # abbiamo 908 valori differenti della superficie

### putting them in different bins to create a categorical feature (library:qcut)

In [ ]:
dfdom.columns

### One hot encoding

In [ ]:
# encode the dx for the following correlation
ohe = OneHotEncoder(
    sparse=False, drop="if_binary", categories="auto"
)  # dummy variable trap - evitata con drop='if_binary'

CNT_TCN_COD = pd.DataFrame(ohe.fit_transform(dfdom["CNT_TCN_COD"].values.reshape(-1, 1)))
SUPERFICIE = pd.DataFrame(
    ohe.fit_transform(dfdom["SUPERFICIE"].values.reshape(-1, 1))
)  # range(istogramma , b8in) quantili
# qcat pandas taglia in quantili
# COMPONENTI=pd.DataFrame(ohe.fit_transform(dfdom['COMPONENTI'].values.reshape(-1, 1)))
NUM_CIR = pd.DataFrame(ohe.fit_transform(dfdom["NUM_CIR"].values.reshape(-1, 1)))
target = pd.DataFrame(ohe.fit_transform(dfdom["target"].values.reshape(-1, 1)))

In [ ]:
# NUM_CIR.columns=['CIR0',  'CIR1',  'CIR2',  'CIR3',  'CIR4',  'CIR5',  'CIR6',  'CIR7',
#                      'CIR8',  'CIR9', 'CIR10', 'CIR11', 'CIR12','CIR13', 'CIR14']

In [ ]:
# NUM_CIR

In [ ]:
df = dfdom

In [ ]:
df

In [ ]:
df["CNT_TCN_COD"] = CNT_TCN_COD

# Ordering "Municipi" by number of "utenze"

In [ ]:
# dfd=pd.read_csv('NDOM_FINAL.csv')
# ord_cir=dfd.groupby('NUM_CIR')[['IMPORTO_CONTRATTO']].agg(
#     ['count']
# )
# ord_cir=ord_cir['IMPORTO_CONTRATTO'].sort_values(by='count',ascending=True).index.values
# ord_cir

In [ ]:
df["NUM_CIR"].replace(
    {
        15.0: 1.0,
        14.0: 2.0,
        13.0: 3.0,
        11.0: 4.0,
        8.0: 5.0,
        6.0: 6.0,
        4.0: 7.0,
        9.0: 8.0,
        12.0: 9.0,
        3.0: 10.0,
        10.0: 11.0,
        5.0: 12.0,
        2.0: 13.0,
        7.0: 14.0,
        1.0: 15.0,
    },
    inplace=True,
)

In [ ]:
# df.columns=[ 'CNT_TCN_COD',  'SUP0',  'SUP1',  'SUP2',  'SUP3',  'COMP0',  'COMP1',  'COMP2',  'COMP3',  'COMP4',
#                     'COMP5',  'COMP6',  'COMP7',  'COMP8',  'COMP9', 'COMP10', 'COMP11',
#                     'COMP12', 'COMP13', 'COMP14', 'COMP15',  'CIR0',  'CIR1',  'CIR2',  'CIR3',  'CIR4',  'CIR5',  'CIR6',  'CIR7',
#                     'CIR8',  'CIR9', 'CIR10', 'CIR11', 'CIR12','CIR13', 'CIR14',  'target']

In [ ]:
df

In [ ]:
features = df

In [ ]:
target = features["target"]

In [ ]:
features = features.drop(columns="target")

In [ ]:
features

# Train test split (stratified)
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.12, random_state=999, stratify=target)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=999, stratify=y_train)
print("Shape of entire dataset: {}".format(str(features.shape)))
print("Shape of train data: {}".format(str(X_train.shape)))
print("Shape of test data: {}".format(str(X_test.shape)))
print("Shape of val data: {}".format(str(X_val.shape)))

In [ ]:
X_train["CNT_TCN_COD"].value_counts()

In [ ]:
df["CNT_TCN_COD"].value_counts()

# Undersampling

In [ ]:
rus = RandomUnderSampler(random_state=0)
# rus.fit(X_train, y_train)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

In [ ]:
X_train

In [ ]:
X_resampled

In [ ]:
len(y_train)

In [ ]:
np.sum(y_train)

In [ ]:
len(y_train) - np.sum(y_train)

In [ ]:
np.sum(y_resampled)

In [ ]:
X_tr = X_resampled

In [ ]:
y_resampled

In [ ]:
X_tr["tar"] = y_resampled

In [ ]:
X_tr = X_tr.sample(frac=1).reset_index(drop=True)

In [ ]:
y_train = X_tr["tar"]

In [ ]:
X_train = X_tr.drop(columns="tar")

In [ ]:
X_train

In [ ]:
y_train

# Feature Importance  SHAP- XGBClassifier

In [ ]:
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)
score = model.score(X_val, y_val)  # e poi validation è anche più grande del test
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="violin")

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="dot")

In [ ]:
dfshap = pd.DataFrame(shap_values)

In [ ]:
X_train.columns

In [ ]:
dfshap.columns = X_train.columns

In [ ]:
for i in range(len(dfshap.columns)):
    print(dfshap.columns[i], ": ", np.sum(dfshap[dfshap.columns[i]]))

In [ ]:
shap_values = explainer.shap_interaction_values(X_train)  # ha solo il dot
shap.summary_plot(shap_values, X_train)

Synergy= the interaction or cooperation of two or more organizations, substances, or other agents to produce a combined effect greater than the sum of their separate effects.

### Analyze it using ExtraTreesClassifier

Using a decison tree with criteria based on Gini impurity for the information gain in order to find the feature's relevance. Where S is the set of items and Si is the the set of items of the i-esim class

Gini index of S $$  GS = 1 - \sum_{i=1}^k ({\frac{\vert S_i \vert }{\vert S \vert}}) ^2 $$

Extremely Randomized Trees Classifier.
This is an ensemble learning technique that output it's classification result aggregating the results of a "forest" of multiple uncorrelated decision trees.
Each tree, at each test node, is provided with a k random sample of features from the feature set.
Each decision tree must select the best feature to split the data based on a mathematical criteria, in our case the Gini Impurity.
This random sample of features leads to the creation of multiple uncorrelated decision trees.
The normalized total reduction in the Gini Impurity, i.e. the Gini Importance of the feature, is computed during the construction of the forest for each feature.
Each feature is ordered according to the Gini Importance of each feature and this allows us to perform the feature selection.

In [ ]:
X = X_train  # .drop(labels=['age','dx','label_dx','dx_vec','color_vector'],axis=1)
y = y_train
# sc = StandardScaler()
# sc = sc.fit(X)
# X_s = pd.DataFrame(sc.transform(X), columns=X.columns)
forest = ExtraTreesClassifier(criterion="gini", n_estimators=250, random_state=0)
X_s = X
# proviamo a vedere intanto cosa esce fuori tenendo solo le features numeriche
# X_s = X_s.apply(pd.to_numeric)#, errors='coerce')
# X_s['MOTIVO_PAREGGIO']=X_s['MOTIVO_PAREGGIO'].replace(np.nan,0.)
# X_s['NUM_RATE']=X_s['NUM_RATE'].replace(np.nan,0.)
# X_s['IMPORTO_RATEIZZATO']=X_s['IMPORTO_RATEIZZATO'].replace(np.nan,0.)
# X_s=X_s.dropna().reset_index(drop=True)#ho controllato, restano nan solo sulle circoscrizioni mancanti
# il problema è che poi non ho modificato di conseguenza anche y
forest.fit(X_s, y)
importances = forest.feature_importances_
ff = np.array([e.feature_importances_ for e in forest.estimators_])
dd = pd.DataFrame(ff, columns=X.columns)
fig = plt.figure(figsize=(16, 6))
sns.barplot(data=dd, ci="sd", alpha=0.7)
plt.xticks(rotation=90)
plt.title("Feature relevance for classification")
plt.show()

## See how the model reacts changing parameters

In [ ]:
forest = ExtraTreesClassifier(criterion="entropy", n_estimators=250, random_state=0)
X_s = X
forest.fit(X_s, y)
importances = forest.feature_importances_
ff = np.array([e.feature_importances_ for e in forest.estimators_])
dd = pd.DataFrame(ff, columns=X.columns)
fig = plt.figure(figsize=(16, 6))
sns.barplot(data=dd, ci="sd", alpha=0.7)
plt.xticks(rotation=90)
plt.title("Feature relevance for classification")
plt.show()

### log_loss does not work!

In [ ]:
forest = ExtraTreesClassifier(criterion="gini", n_estimators=50, random_state=0)
X_s = X
forest.fit(X_s, y)
importances = forest.feature_importances_
ff = np.array([e.feature_importances_ for e in forest.estimators_])
dd = pd.DataFrame(ff, columns=X.columns)
fig = plt.figure(figsize=(16, 6))
sns.barplot(data=dd, ci="sd", alpha=0.7)
plt.xticks(rotation=90)
plt.title("Feature relevance for classification")
plt.show()

In [ ]:
# forest = ExtraTreesClassifier(criterion = 'gini', n_estimators=500, random_state=0)
# X_s=X
# forest.fit(X_s, y)
# importances = forest.feature_importances_
# ff = np.array([e.feature_importances_ for e in forest.estimators_])
# dd = pd.DataFrame(ff, columns=X.columns)
# fig = plt.figure(figsize=(16, 6))
# sns.barplot(data=dd, ci="sd", alpha=.7)
# plt.xticks(rotation=90)
# plt.title('Feature relevance for classification')
# plt.show()

In [ ]:
# forest = ExtraTreesClassifier(criterion = 'gini', n_estimators=10, random_state=0)
# X_s=X
# forest.fit(X_s, y)
# importances = forest.feature_importances_
# ff = np.array([e.feature_importances_ for e in forest.estimators_])
# dd = pd.DataFrame(ff, columns=X.columns)
# fig = plt.figure(figsize=(16, 6))
# sns.barplot(data=dd, ci="sd", alpha=.7)
# plt.xticks(rotation=90)
# plt.title('Feature relevance for classification')
# plt.show()

In [ ]:
# forest = ExtraTreesClassifier(criterion = 'entropy', n_estimators=10, random_state=0)
# X_s=X
# forest.fit(X_s, y)
# importances = forest.feature_importances_
# ff = np.array([e.feature_importances_ for e in forest.estimators_])
# dd = pd.DataFrame(ff, columns=X.columns)
# fig = plt.figure(figsize=(16, 6))
# sns.barplot(data=dd, ci="sd", alpha=.7)
# plt.xticks(rotation=90)
# plt.title('Feature relevance for classification')
# plt.show()

In [ ]:
# X_t=X_train

In [ ]:
# X_t['target']=y_train

In [ ]:
# explainer = lime_tabular.LimeTabularExplainer(
#     training_data=np.array(X_train),
#     feature_names=X_train.columns,
#     class_names=['target'],
#     mode='classification'
# )

In [ ]:
# # define dataset
# # X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)
# # define the model
# model = LogisticRegression()

# model.fit(X_train, y_train)
# score = model.score(X_test, y_test)

In [ ]:
# X_train=X_train.drop(columns='target')

In [ ]:
# exp = explainer.explain_instance(
#     data_row=X_train.iloc[1],
#     predict_fn=model.predict_proba
# )

# exp.show_in_notebook(show_table=True)


## Tree models are not working well with OHE!


Tree-based models, such as Decision Trees, Random Forests, and Boosted Trees, typically don't perform well with one-hot encodings with lots of levels. This is because they pick the feature to split on based on how well that splitting the data on that feature will "purify" it. If we have a lot of levels, only a small fraction of the data (typically) will belong to any given level, so the one-hot encoded columns will be mostly zeros. Since splitting on this column will only produce a small gain, tree-based algorithms typically ignore the information in favor of other columns. This problem persists, regardless of the volume of data you actually have.

###  Mutual Info Classification to measure the dependency between the variables

Mutual Info Classification in order to measure the dependency between the variables. It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency. The function relies on nonparametric methods based on entropy estimation:
\begin{equation}
    Entropy(x) = - \sum_{x} {p(x) log_2 p(x)}
\end{equation}

# Logistic Regression  Feature Importance

In [ ]:
# define dataset
# X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)
# define the model
model = LogisticRegression()
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.coef_[0]
# summarize feature importance
for i, v in enumerate(importance):
    print("Feature: %s, Score: %.5f" % (X_train.columns[i], v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
pd.Series(X_train.columns).astype(str)

In [ ]:
df_imp = pd.DataFrame()
df_imp["features"] = X_train.columns
df_imp["importance"] = np.abs(importance)
df_imp.sort_values(by="importance", ascending=False)

In [ ]:
# define the model
model = LogisticRegression()
# fit the model
model.fit(X_train, y_train)

variance = np.diagonal(np.linalg.inv(np.dot(X_train.T, X_train)) * model.coef_.T.dot(model.coef_))
print(variance)

In [ ]:
df_imp = pd.DataFrame()
df_imp["features"] = X_train.columns
df_imp["importance"] = np.abs(importance)
df_imp["variance"] = variance

In [ ]:
df_imp.sort_values(by="importance", ascending=False)

In [ ]:
# Xs = add_constant(X_train)

In [ ]:
# Xs

In [ ]:
# model = LogisticRegression().fit(Xs, y_train)

In [ ]:
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(Xs.values, i) for i in range(Xs.shape[1])]
# vif["features"] = Xs.columns

In [ ]:
# print(vif)
#  VIF > 5 indicate high multicollinearity, which means they are
# highly correlated with each other and may be contributing to the
# variance in the model. You can drop these features or try to address
# the multicollinearity by combining them into a single feature.

## Using statsmodels

In [ ]:
# Fit the logistic regression model
model = sm.Logit(y_train, X_train).fit()

# Print the summary output
print(model.summary())  # i coefficienti sono gli stessi

# Probit

In [ ]:
model = sm.Probit(y_train, X_train).fit()

# Print the summary output
print(model.summary())  # i coefficienti sono gli stessi

# CART Feature Importance(DecisionTreeClassifier)

In [ ]:
# decision tree for feature importance on a classification problem
from matplotlib import pyplot

# define the model
model = DecisionTreeClassifier()
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i, v in enumerate(importance):
    print("Feature: %s, Score: %.5f" % (X_train.columns[i], v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
df_imp = pd.DataFrame()
df_imp["features"] = X_train.columns
df_imp["importance"] = importance
df_imp.sort_values(by="importance", ascending=False)

#  Random Forest Classification Feature Importance

the importance of features calculated using the variance measure in Random Forest may not be the same as the actual predictive power of the features. 

In [ ]:
# random forest for feature importance on a classification problem

# define the model
model = RandomForestClassifier()
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i, v in enumerate(importance):
    print("Feature: %s, Score: %.5f" % (X_train.columns[i], v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
df_imp = pd.DataFrame()
df_imp["features"] = X_train.columns
df_imp["importance"] = importance
df_imp.sort_values(by="importance", ascending=False)

# xgboost feature importance

In [ ]:
# check xgboost version
print(xgboost.__version__)

In [ ]:
# xgboost for feature importance on a classification problem

from matplotlib import pyplot

# define dataset
# X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)
# define the model
model = XGBClassifier()
# fit the model
model.fit(X_train, y_train)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i, v in enumerate(importance):
    print("Feature: %s, Score: %.5f" % (X_train.columns[i], v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
df_imp = pd.DataFrame()
df_imp["features"] = X_train.columns
df_imp["importance"] = importance
df_imp.sort_values(by="importance", ascending=False)

## Logistic Regression


Logistic Regression is a Generalized Linear Model (GLM) such that:


$$ P(C_{1}|\textbf{x}) = \frac{1}{1 - e^{\textbf{w}^{T}\phi(\textbf{x})}}$$

We tried to improve results obtained trought Naive Bayes approach using this slightly more sophisticated generative method trusting in an improvement. Being that the logistic regression model can be considered as the equivalent, for binary classification, of linear regression for the regression case it is still a simple method. 
Anyeway it is not only a classification model, but also gives back  probabilities. This is a big advantage over models that can only provide the final classification beacuse it is more informative.





We decided to train the model with imputed and standardized data, in particular we decided to impute the model by replacing the values marked as -999,000 with the most frequent value relative to the particular feature. Comparing the results found on the internet relating to this type of classifiers it proved to be the method that guarantees the best results.

Split the data for validation

Using sklearn.linear_model.LogisticRegression funtion we maneged to create and fit the model.
The fit consist in maximizing log-likehood function dependent on d + 1 parameters $\textbf{w}=(w_{0},...,w_{d})$

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

Parameters $\textbf{w}=(w_{1},...,w_{d})$ are stored in

In [ ]:
print("predicted: ", logreg.coef_)

While intercept value $w_{0}$

In [ ]:
print("predicted: ", logreg.intercept_)

Once the model is optimized, it's possible to use it to evaluete test set 

In [ ]:
y_val_pred = logreg.predict(X_val)
prob_predict_val = logreg.predict_proba(X_val)
print(
    "Accuracy of Logistic Regression classifier on validation set: {:.3f}".format(
        cross_val_score(logreg, X_val, y_val, cv=5, scoring="accuracy").mean()
    )
)

The function assign to singal or backgroung class the element with predicted probabiility pending respectively towards one or zero.

In [ ]:
print("predicted: ", y_val_pred)

In [ ]:
print("predicted: ", prob_predict_val)

In [ ]:
confusion_matrix = confusion_matrix(y_val, y_val_pred)
print(confusion_matrix)

It's interesting interpolate the AUC-ROC curve to visualize the performance of a classifier method. It is a performance measurement for classification problems at various thresholds settings. ROC is a probability curve and AUC represents degree or measure of separability. It tells how much model is capable of distinguishing between classes.

In [ ]:
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:, 1])
plt.figure()
plt.plot(fpr, tpr, label="Logistic Regression (area = %0.2f)" % logit_roc_auc)
plt.plot([0, 1], [0, 1], "r--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.savefig("Log_ROC")
plt.show()